In [1]:
#!pwd
#!source activate sed_env

In [1]:
!pwd
import pandas as pd
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import splat
import astropy.units as u
from sedkit import SED
import astropy.units as u
from bokeh.io import output_notebook
from bokeh.plotting import show
from pkg_resources import resource_filename
output_notebook()
#%matplotlib inline

/Users/caganze/research/J1624/notebooks
Adding 89 sources from /Users/caganze/research/splat//resources/Spectra/Public/MAGE/ to spectral database
Adding 145 sources from /Users/caganze/research/splat//resources/Spectra/Public/LRIS-RED/ to spectral database
Adding 2404 sources from /Users/caganze/research/splat//resources/Spectra/Public/SPEX-PRISM/ to spectral database


ModuleNotFoundError: No module named 'sedkit'

In [2]:
import sys
sys.version

'3.8.5 (default, Sep  4 2020, 02:22:02) \n[Clang 10.0.0 ]'

In [3]:
data_folder='/users/caganze/research/J1624/data/'
df=pd.read_csv(data_folder+'/j1624_kast_nires.csv')

In [4]:
castar = SED(name='J1624')
castar.ra=246.0684208*u.deg
castar.dec=-32.2037824*u.deg
#spectral type
castar.spectral_type = (64.5, 1.4)
castar.parallax=(59.01*u.mas, 0.12*u.mas)
#wise photometry
castar.add_photometry('WISE.W1', 7.936, 0.026)
castar.add_photometry('WISE.W2', 7.768, 0.020)
castar.add_photometry('WISE.W3', 7.650, 0.018)
castar.add_photometry('WISE.W4', 7.329, 0.154)
#2mass photometry
castar.add_photometry('2MASS.J', 9.150, 0.026)
castar.add_photometry('2MASS.H', 8.470, 0.029)
#castar.add_photometry('2MASS/2MASS.K', 8.154, 0.023) not in database?

#gaia photometry
#castar.add_photometry('Gaia.G', 12.6265, 0.0139)
#castar.add_photometry('Gaia.bp', 14.2129, 0.0045)
#castar.add_photometry('Gaia.rp', 11.0375, 0.0030)

#add optical V band photometry
#castar.add_photometry('PAN-STARRS/PS1.g', 14.6207, 0.0012)

[sedkit] SED initialized
[sedkit] Setting name to J1624
[sedkit] Setting age to (<Quantity 6. Gyr>, <Quantity 4. Gyr>) with reference 'None'
[sedkit] Setting sky_coords to <SkyCoord (ICRS): (ra, dec) in deg
    (246.0684208, -32.2037824)>
[sedkit] There was a problem determining the interstellar reddening. Setting to 0. You can manually set this with the 'reddening' attribute.
[sedkit] Setting radius to (<Quantity 0.16 solRad>, <Quantity 0.01 solRad>) with reference 'None'
[sedkit] Setting spectral_type to (64.5, 1.4, 'V', None, None) with reference 'None'
[sedkit] Setting parallax to (<Quantity 59.01 mas>, <Quantity 0.12 mas>) and distance to (<Quantity 16.95 pc>, <Quantity 0.03 pc>, <Quantity 0. pc>) with reference 'None'
[sedkit] Setting WISE.W1 photometry to 7.936 (0.026) with reference 'None'
[sedkit] Setting WISE.W2 photometry to 7.768 (0.02) with reference 'None'
[sedkit] Setting WISE.W3 photometry to 7.65 (0.018) with reference 'None'
[sedkit] Setting WISE.W4 photometry to 7.32

In [5]:
castar.find_SDSS()

[sedkit] 0 records found in V/147.


In [6]:
#find v band photometry
castar.find_Simbad()


In [18]:
s=splat.Spectrum(wave=df['wave (micron)'].values*u.micron,
                flux=df['flux (funits)'].values*u.erg/u.micron/u.s/(u.cm**2),
                noise=df['noise'].values*u.erg/u.micron/u.s/(u.cm**2))
#add a mask
mask=df['mask']
#MASK OUT OPTICAL SPECTRUM
#mask=np.logical_or(mask, s.wave.value<1.0)
from sedkit import Spectrum
spec = Spectrum(s.wave[~mask], s.flux[~mask], s.noise[~mask], name='J1624')

In [19]:
#castar.add_spectrum(spec)

In [20]:
# Fit to infer spectral type
castar.fit_spectral_type()
# Fit to infer fundamental params
from sedkit import BTSettl
castar.fit_modelgrid(BTSettl())
#castar.make_sed
castar.fit_blackbody()

[sedkit] Best fit SpeX Prism Library: Opt:M4V
[sedkit] Best fit BT-Settl: 3000.0/5.5/0.0/0.0
[sedkit] Blackbody fit: 2645 K


In [21]:
castar.results

param,value,unc,units
object,object,object,object
name,J1624,--,--
ra,246.0684208,--,--
dec,-32.2037824,--,--
age,6.0,4.0,Gyr
membership,--,--,--
distance,16.95,0.03,pc
parallax,59.01,0.12,mas
SpT,M4.5V,--,--
spectral_type,64.5,1.4,--


In [22]:
from bokeh.plotting import  output_file, save
f=castar.plot(integral=True, best_fit=True, one_color=False)

In [23]:
#castar.results

In [24]:
#from astropy.coordinates import SkyCoord
#splat.coordinateToDesignation(SkyCoord(ra=castar.ra*u.deg, dec=castar.dec*u.deg))
#add spectrum and redo fits 
castar.add_spectrum(spec)
castar.fit_spectral_type()
castar.fit_modelgrid(BTSettl())

[sedkit] Spectrum added.
[sedkit] Best fit SpeX Prism Library: Opt:L1
[sedkit] Best fit BT-Settl: 2100.0/5.5/0.0/0.0


In [25]:
f=castar.plot(integral=True, best_fit=True, one_color=False)


In [33]:
#mask
#mask=np.logical_or(mask, s.wave.value<1.0)
castar.drop_spectrum(-1)
mask=np.logical_or(mask, s.wave.value<1.0)
spec = Spectrum(s.wave[~mask], s.flux[~mask], s.noise[~mask], name='J1624')
castar.fit_spectral_type()
castar.fit_modelgrid(BTSettl())

[sedkit] Best fit SpeX Prism Library: Opt:M4V
[sedkit] Best fit BT-Settl: 3000.0/5.5/0.0/0.0


In [34]:
fx=castar.plot(integral=True, best_fit=True, one_color=False)


In [43]:
#castar.drop_spectrum(-1)
mask=np.logical_and(s.wave.value <1.0, s.wave.value>0.6)
spec = Spectrum(s.wave[mask], s.flux[mask], s.noise[mask], name='J1624')
castar.fit_spectral_type()
castar.fit_modelgrid(BTSettl())

[sedkit] Could not fit model grid SpeX Prism Library to the 'spec' SED. No spectrum to fit.
[sedkit] Could not fit model grid BT-Settl to the 'spec' SED. No spectrum to fit.


In [44]:
fx=castar.plot(integral=True, best_fit=True, one_color=False)
